# 1. Imports

In [8]:
from analyze import plot_metric

# 2. Descriptive Analytics

In [9]:
csv_file = './data/survey_pre_processed_data.csv'

In [10]:
plot_metric(csv_file,'Q10_2', 'interest in V2G', 'count of participants')

TypeError: plot_metric() takes from 1 to 3 positional arguments but 4 were given